In [2]:
import pandas as pd
import itertools as it
from collections import defaultdict,Counter
import numpy as np

c:\Users\ignac\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ignac\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\ignac\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load Loop-Motif Intersection Data (Script 1)

Script is run once per sample. Output a tsv of results.

In [3]:
columns = ['loop_anchor1_chr','loop_anchor1_start','loop_anchor1_end','loop_anchor2_chr','loop_anchor2_start','loop_anchor2_end','motif_chr','motif_start','motif_end','motif_id','motif_name']
df = pd.read_csv(r"C:\Users\ignac\Downloads\loops_overlap_motifs.txt",sep="\t",header =None,names=columns)

# Get rid of all duplicate motif-pair
# (doesn't remove dupliate motifs in a single anchort)
df = df.drop_duplicates().reset_index(drop=True)

In [4]:
df["anchor_1"] = (df['loop_anchor1_start'] <= df['motif_start']) & (df['motif_end'] <= df['loop_anchor1_end']) 
df["anchor_2"] = (df['loop_anchor2_start'] <= df['motif_start']) & (df['motif_end'] <= df['loop_anchor2_end'])
df['anchor_id'] = 1

df.loc[df['anchor_1']==0,'anchor_id'] = 2
df = df.drop_duplicates().reset_index(drop=True)
df['anchor1_id'] = df['loop_anchor1_chr'] + ':' + df['loop_anchor1_start'].astype(str)
df['anchor2_id'] = df['loop_anchor2_chr'] + ':' + df['loop_anchor2_start'].astype(str)
df['loop_id'] = df['loop_anchor1_chr'] + ':' + df['loop_anchor1_start'].astype(str) + ':' + df['loop_anchor2_start'].astype(str)

In [8]:
df.drop_duplicates(subset=['loop_id', 'anchor1_id', 'motif_id'], inplace=True)
df.drop_duplicates(subset=['loop_id', 'anchor2_id', 'motif_id'], inplace=True)

## Count Motif-Pairs in Our Dataset (script 1)

In [9]:
motif_pair_counter = Counter()

In [10]:
for loop_id, loop_df in df.groupby('loop_id'):
    
    anchor1_df = loop_df.loc[loop_df.anchor_1 == True]
    anchor2_df = loop_df.loc[loop_df.anchor_2 == True]
    
    if anchor1_df.shape[0] == 0 or anchor2_df.shape[0] == 0:
        continue
        
    motifs1 = list(anchor1_df.motif_name)
    motifs2 = list(anchor2_df.motif_name)
    perms = list(it.product(motifs1, motifs2))
    
    for p in perms:
        motif_pair_counter[p] += 1


## Bootstrap Test (Genome-Wide) (Script 2)

Run once per each simulation. Use as input, the results from script 1. Output a column of data where each entry is the number of simulated counts

In [64]:
all_loop_pairs = []
i = 0
boostrap_df = df
##Permutate All motifs across the Chromosome
sim = 1
results = []
##Create Simulations
while sim < 3:
    #Randomly sort out permutation column with motifs
    boostrap_df["Random_Motif"] = np.random.permutation(df['motif_name'])

    #Call in counter function
    sim_motif_pair_counter = Counter()
    
    #Groupby loop id, determine anchors based on positoin
    for loop_id, loop_df in df.iloc[0:5000].groupby('loop_id'):
        anchor1_df = loop_df.loc[loop_df.anchor_1 == True]
        anchor2_df = loop_df.loc[loop_df.anchor_2 == True]

        #If missing anchors, skip loop
        if anchor1_df.shape[0] == 0 or anchor2_df.shape[0] == 0:
            continue

        #Get motifs1
        motifs1 = list(anchor1_df.Random_Motif)

        #Get motifs2
        motifs2 = list(anchor2_df.Random_Motif)

        # get motif-pair permutations the combinations
        perms = list(it.product(motifs1, motifs2))
        combs = list(set([sorted(x) for x in perms]))

        # count each combination
        for c in combs:
            sim_motif_pair_counter[c] += 1
    results.append(sim_motif_pair_counter)
    print(sim)
    sim += 1

TypeError: unhashable type: 'list'

In [65]:
(perms[0])

('HOXA10', 'ZNF148')

## Aggregate the simulation data (Script 3)

In [46]:
# reformat the counter
d = defaultdict(lambda: np.zeros(len(results)))
for num in range(0,len(results)):
    for element, count in results[num].items():
        d[element][num] += count

c = Counter(d)

In [47]:
c

Counter({('STAT1::STAT2', 'FOXO1::ELK1'): array([2., 1.]),
         ('PLAG1', 'FOXO1::ELK1'): array([1., 1.]),
         ('PRDM9', 'FOXO1::ELK1'): array([4., 0.]),
         ('ZNF384', 'FOXO1::ELK1'): array([2., 0.]),
         ('ZNF449', 'FOXO1::ELK1'): array([2., 0.]),
         ('KLF3', 'FOXO1::ELK1'): array([1., 0.]),
         ('ZNF93', 'FOXO1::ELK1'): array([1., 1.]),
         ('EGR1', 'FOXO1::ELK1'): array([1., 0.]),
         ('TCF4', 'FOXO1::ELK1'): array([1., 0.]),
         ('ZEB1', 'FOXO1::ELK1'): array([1., 0.]),
         ('ZNF281', 'FOXO1::ELK1'): array([2., 0.]),
         ('CTCFL', 'FOXO1::ELK1'): array([2., 0.]),
         ('ZNF460', 'FOXO1::ELK1'): array([4., 0.]),
         ('ESR2', 'FOXO1::ELK1'): array([1., 0.]),
         ('SPIB', 'ZNF135'): array([2., 1.]),
         ('TCF12', 'ZNF135'): array([2., 1.]),
         ('PATZ1', 'ZBTB6'): array([3., 0.]),
         ('PATZ1', 'GLIS1'): array([1., 0.]),
         ('PATZ1', 'SP5'): array([7., 5.]),
         ('ZNF148', 'ZBTB6'): array([

In [ ]:
# 

In [48]:
t = 0
mp_sigs = []
for motif in motif_pair_counter.items():
    #Get p-value 
    if np.count_nonzero(c[motif[0]]) > 0:
        pval = sum(i <= motif_pair_counter[motif[0]] for i in c[motif[0]] )/len(results)
        if pval >= 0.95:
            print(str(motif[0]) + " Pair Significant at 95% Level")
            t+=1
        mp_sigs.append([motif[0], pval])
    else:
        mp_sigs.append([motif[0], np.nan])
        
print(str(t) + " Motif Pairs Out of " + str(len(motif_pair_counter))+ " Statistically Significant")

('ZNF768', 'RREB1') Pair Significant at 95% Level
('SP8', 'RREB1') Pair Significant at 95% Level
('SP9', 'RREB1') Pair Significant at 95% Level
('SP3', 'RREB1') Pair Significant at 95% Level
('PATZ1', 'RREB1') Pair Significant at 95% Level
('HES1', 'RREB1') Pair Significant at 95% Level
('KLF17', 'RREB1') Pair Significant at 95% Level
('MAZ', 'RREB1') Pair Significant at 95% Level
('KLF17', 'FOXJ2::ELF1') Pair Significant at 95% Level
('KLF17', 'KLF4') Pair Significant at 95% Level
('KLF17', 'KLF15') Pair Significant at 95% Level
('KLF17', 'ZNF454') Pair Significant at 95% Level
('KLF17', 'CTCF') Pair Significant at 95% Level
('KLF17', 'CTCFL') Pair Significant at 95% Level
('KLF17', 'ZKSCAN5') Pair Significant at 95% Level
('KLF17', 'ZNF263') Pair Significant at 95% Level
('KLF17', 'ZNF281') Pair Significant at 95% Level
('KLF17', 'ZNF148') Pair Significant at 95% Level
('KLF17', 'SP5') Pair Significant at 95% Level
('KLF17', 'PRDM9') Pair Significant at 95% Level
('KLF17', 'ZNF530') 

In [49]:
mp_sigs = pd.DataFrame(mp_sigs)
mp_sigs.columns = ['motif-pair', 'p-val']

In [52]:
mp_sigs

,motif-pair,p-val
0,"(ZNF768, RREB1)",1.0
1,"(SP8, RREB1)",1.0
2,"(SP9, RREB1)",1.0
3,"(SP3, RREB1)",1.0
4,"(PATZ1, RREB1)",1.0
...,...,...
123145,"(E2F8, STAT1::STAT2)",NaN
123146,"(E2F8, IRF2)",NaN
123147,"(E2F8, PRDM4)",NaN
123148,"(SOX10, IRF2)",NaN


In [ ]:
for loop_id, loop_df in df.groupby('loop_id'):
    
    anchor1_df = loop_df.loc[loop_df.anchor_1 == True]
    anchor2_df = loop_df.loc[loop_df.anchor_2 == True]
    
    if anchor1_df.shape[0] == 0 or anchor2_df.shape[0] == 0:
        continue
        
    motifs1 = list(anchor1_df.motif_name)
    motifs2 = list(anchor2_df.motif_name)
    perms = list(it.product(motifs1, motifs2))
    
    for p in perms:
        motif_pair_counter[p] += 1
